# 01 - The Ariadne Advantage: Intelligent Routing Demonstration

This notebook demonstrates Ariadne's core feature: the intelligent quantum router. The router analyzes circuit properties, specifically entanglement, to dynamically select the optimal simulation backend (e.g., Matrix Product State (MPS) for low entanglement, or a dense State Vector backend for high entanglement).

We will define two circuits and use the router to see the **Prioritized Filter Chain** in action.

In [ ]:
!pip install git+https://github.com/Hmbown/ariadne.git --quiet

In [ ]:
import numpy as np
from qiskit import QuantumCircuit

from ariadne import explain_routing, simulate

## Case 1: Low Entanglement Circuit (MPS Candidate)

We create a large circuit (N=25) with shallow depth and only nearest-neighbor CNOTs. This structure typically results in low entanglement, making it an ideal candidate for the MPS backend, which scales polynomially with the number of qubits.

In [ ]:
# Low Entanglement Circuit (N=25, shallow depth, local gates)
N_LOW = 25
qc_low = QuantumCircuit(N_LOW)
for i in range(N_LOW):
    qc_low.h(i)
for i in range(0, N_LOW - 1, 2):
    qc_low.cx(i, i + 1)
qc_low.measure_all()

print(f"Low Entanglement Circuit: {N_LOW} qubits, {qc_low.depth()} depth")
print("Estimated entanglement: LOW (nearest-neighbor only)")
print("Routing explanation:")
print(explain_routing(qc_low))

In [ ]:
# Simulate the circuit
result_low = simulate(qc_low, shots=100)

print("\n--- Low Entanglement Circuit Routing ---")
print(f"✅ Backend Selected: {result_low.backend_used.value}")
print(f"⏱️  Execution Time: {result_low.execution_time:.4f}s")
print(f"🎯 Confidence Score: {result_low.routing_decision.confidence_score:.3f}")
print(f"📊 Throughput: {100/result_low.execution_time:.0f} shots/sec")
print(f"\nSample results (first 5 states): {dict(list(result_low.counts.items())[:5])}")

## Case 2: High Entanglement Circuit (Dense State Vector Candidate)

We create a smaller circuit (N=10) but with deep, highly non-local entanglement layers. This structure quickly maximizes entanglement, making the MPS approximation inefficient. The router should reject MPS and default to the fastest dense state vector backend available (e.g., Qiskit).

In [ ]:
# High Entanglement Circuit (N=10, deep, highly non-local)
N_HIGH = 10
qc_high = QuantumCircuit(N_HIGH)
for _ in range(5):  # 5 layers of entanglement
    for i in range(N_HIGH):
        qc_high.rx(np.pi / 4, i)
    # Highly entangling layer
    for i in range(N_HIGH - 1):
        qc_high.cz(i, i + 1)
    qc_high.barrier()
qc_high.measure_all()

print(f"High Entanglement Circuit: {N_HIGH} qubits, {qc_high.depth()} depth")
print("Estimated entanglement: HIGH (deep, non-local layers)")
print("Routing explanation:")
print(explain_routing(qc_high))

In [ ]:
# Simulate the circuit
result_high = simulate(qc_high, shots=100)

print("\n--- High Entanglement Circuit Routing ---")
print(f"✅ Backend Selected: {result_high.backend_used.value}")
print(f"⏱️  Execution Time: {result_high.execution_time:.4f}s")
print(f"🎯 Confidence Score: {result_high.routing_decision.confidence_score:.3f}")
print(f"📊 Throughput: {100/result_high.execution_time:.0f} shots/sec")
print(f"\nSample results (first 5 states): {dict(list(result_high.counts.items())[:5])}")

## Summary

As you can see, Ariadne's router correctly:

1. **Analyzes circuit structure** to determine entanglement characteristics
2. **Selects optimal backend** based on circuit properties
3. **Provides transparent routing** with explanations
4. **Achieves good performance** on both low and high entanglement circuits

The router automatically handles:
- **Clifford circuits** → Stim backend (fast stabilizer simulator)
- **Low-entanglement** → MPS backend (matrix product states)
- **General circuits** → Qiskit backend (reliable fallback)

This means you don't need to worry about which backend to use - just write your quantum circuit and Ariadne handles the rest!